In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
import re
import bql
import bqviz as bqv
from bqplot import pyplot as plt
from pandas.tseries.offsets import BDay
import functools

today = pd.datetime.today()
startrange = today - BDay(31)
endrange = today - BDay(1)
startrange = startrange.strftime('%Y-%m-%d')
endrange = endrange.strftime('%Y-%m-%d')


In [2]:
bq = bql.Service()


CUSIPDB = pd.read_excel('CUSIP Database Sectors.xlsx')


sec_name = bq.data.SECURITY_DES()
MTG_CMO_CLASS = bq.data.MTG_CMO_CLASS()
price = bq.data.px_last(start=startrange,end=endrange,per='D')


sec_name_response = bq.execute(bql.Request(CUSIPDB['Identifier'][0:5000,],sec_name))
sec_name_response2 = bq.execute(bql.Request(CUSIPDB['Identifier'][5000:len(CUSIPDB['Identifier']),],sec_name))
MTG_CMO_CLASSresponse = bq.execute(bql.Request(CUSIPDB['Identifier'][0:5000,],MTG_CMO_CLASS))
MTG_CMO_CLASSresponse2 = bq.execute(bql.Request(CUSIPDB['Identifier'][5000:len(CUSIPDB['Identifier']),],MTG_CMO_CLASS))
priceresponse = bq.execute(bql.Request(CUSIPDB['Identifier'][0:5000,],price))
priceresponse2 = bq.execute(bql.Request(CUSIPDB['Identifier'][5000:5152,],price))



In [3]:
price = bq.data.px_last(start=startrange,end=endrange,per='D')

pricedf = priceresponse[0].df().drop('CURRENCY',axis=1).reset_index().set_index('DATE').groupby('DATE').apply(lambda g: g.set_index('ID').unstack()).T
pricedf = pricedf.reset_index().drop('level_0',axis=1).set_index('ID')
pricedf = pricedf.dropna(axis='columns',how='all')

In [108]:
pricedftwo = priceresponse2[0].df().drop('CURRENCY',axis=1).reset_index().set_index('DATE').groupby('DATE').apply(lambda g: g.set_index('ID').unstack()).T
pricedftwo = pricedftwo.reset_index().drop('level_0',axis=1).set_index('ID')
pricedftwo = pricedftwo.dropna(axis='columns',how='all')


In [ ]:
masterprice = pd.concat([pricedf,pricedftwo])

In [167]:
MasterDF = pd.concat([sec_name_response[0].df(),sec_name_response2[0].df()])
MasterDF = pd.concat([MasterDF,pd.concat([MTG_CMO_CLASSresponse[0].df(),MTG_CMO_CLASSresponse2[0].df()])],axis=1)
MasterDF.columns = ['Sec_Desc','Class']
MasterDF = pd.concat([MasterDF,CUSIPDB.set_index('Identifier')],axis=1)

In [168]:
MasterDF = pd.concat([MasterDF,masterprice],axis=1,sort=False)

In [169]:
MasterDF.columns.values[3:len(MasterDF.columns)] = [datetime.strftime(item, '%m/%d/%Y') for item in MasterDF.columns.values[3:len(MasterDF.columns)]]

In [174]:
MasterDF = MasterDF.dropna(subset = ['Sec_Desc'])

In [175]:
MasterDF = MasterDF.reset_index(drop=True)
MasterDF = MasterDF.set_index('Sec_Desc')

In [ ]:
NewDF = MasterDF.iloc[:,2:len(MasterDF.columns)].T
NewDF.index = pd.DatetimeIndex(NewDF.index)

#Historical BVAL Pricing (31 Day Period)

In [179]:
# Pass the example DataFrame to the bqviz LinePlot object and apply Bloomberg styles
#bqv_plot = bqv.InteractiveLinePlot(MasterDF.iloc[:,3:len(MasterDF.columns)].T)
#graph_DF = MasterDF.iloc[:,2:len(MasterDF.columns)].T

bqv_plot = bqv.InteractiveLinePlot(NewDF)
bqv_plot.xaxis.tick_rotate = 90
bqv_plot.height = '50000'
bqv_plot.figure.layout = {'width':'1500px', 'height':'1500px'}
bqv_plot.xaxis.tick_style = {'font-size': 12}
# Display the plot
bqv_plot.show()